In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import time

In [0]:
df = spark.read.format("parquet")\
  .load("abfss://bronze@project1ds.dfs.core.windows.net/customers")

In [0]:
df.display()

In [0]:
df.printSchema()

In [0]:
df =df.drop('_rescued_data')
df.display()

In [0]:
#Check for nulls and duplicates
dq_report = df.select(
    count(when(col("customer_id").isNull(), 1)).alias("null_order_id"),
    count("*").alias("total_rows"),
)

dup_report = (df.groupBy("customer_id")
                .agg(count("*").alias("cnt"))
                .filter(col("cnt") > 1)
                .count())

dq_report.show()
print(f"Duplicate order_id count: {dup_report}")

In [0]:
df = df.withColumn("domains", split(col("email"),"@")[1])
df.display()

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort("total_customers",ascending=False).display()

In [0]:
df_gmail = df.filter(col("domains") == "gmail.com")
df_gmail.display()
time.sleep(5)

df_yahoo = df.filter(col("domains") == "yahoo.com")
df_gmail.display()
time.sleep(5)

df_hotmail = df.filter(col("domains") == "hotmail.com")
df_gmail.display()
time.sleep(5)

In [0]:
df = df.withColumn("full_name",concat(col("first_name"),lit(" "),col("last_name")))
df=df.drop('first_name','last_name')
df.display()

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@project1ds.dfs.core.windows.net/customers")

In [0]:
%sql
create schema project1_catalog.silver

In [0]:
%sql
create table if not exists project1_catalog.silver.customers_silver
using delta
location "abfss://silver@project1ds.dfs.core.windows.net/customers"

In [0]:
%sql
select * from project1_catalog.silver.customers_silver;